# Saga Falabella

In [49]:
from selenium import webdriver                                # importa la clase principal para controlar el navegador
from selenium.webdriver.chrome.options import Options          # importa la clase para configurar opciones de Chrome
from selenium.webdriver.common.by import By                    # importa los localizadores (By.ID, By.CSS_SELECTOR, etc.)
from selenium.webdriver.common.keys import Keys                # importa teclas especiales (END, ENTER, etc.)
from selenium.webdriver.support.ui import WebDriverWait        # importa la utilidad para esperas explícitas
from selenium.webdriver.support import expected_conditions as EC  # importa condiciones esperadas para WebDriverWait
import time                                                   # importa time para sleeps sencillos


def scrape_falabella_selenium(query, max_scrolls=10, max_results=50):
    # función principal que recibe:
    # - query: texto a buscar en Falabella,
    # - max_scrolls: número máximo de "END" (scrolls) para forzar carga de más productos,
    # - max_results: límite de resultados que queremos devolver.
    print(f"\n🛍️ Resultados de Falabella para: {query}")       # imprime en consola el término que se está buscando

    options = Options()                                        # crea un objeto de opciones para Chrome
    options.add_argument("--headless=new")                     # ejecuta Chrome en modo headless (sin UI). Quita si quieres ver el navegador.
    options.add_argument("--start-maximized")                  # inicia la ventana maximizada (útil para que ciertos sitios carguen vistas completas)

    driver = webdriver.Chrome(options=options)                 # instancia el driver de Chrome con las opciones definidas
    resultados = []                                            # lista vacía donde iremos guardando los diccionarios de cada producto

    try:
        url = f"https://www.falabella.com.pe/falabella-pe/search?Ntt={query.replace(' ', '+')}"
        # construye la URL de búsqueda reemplazando espacios por '+' para formar la query string
        driver.get(url)                                        # carga la página en el navegador controlado por Selenium

        # espera explícita: hasta que haya al menos un elemento con selector "a.pod-link"
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.pod-link"))
        )

        prev_count = 0                                         # contador previo para comparar si aparecen más productos después de cada scroll
        for i in range(max_scrolls):                           # bucle que hará hasta max_scrolls scrolls para forzar la carga lazy
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
            # envia la tecla END al body (equivale a scrollear hasta abajo)
            time.sleep(3)                                      # espera fija corta para que el sitio cargue más elementos después del scroll

            productos_actuales = driver.find_elements(By.CSS_SELECTOR, "a.pod-link")
            # busca todos los nodos 'a' con clase pod-link (cada uno representa un producto en el listado)
            print(f"🔄 Scroll {i+1}: {len(productos_actuales)} productos detectados")
            # imprime cuántos productos hay tras este scroll (útil para depuración)

            if len(productos_actuales) == prev_count:         # si el número no creció desde el último scroll...
                break                                         # ...no hay más productos cargándose -> salimos del bucle
            prev_count = len(productos_actuales)               # actualizamos el contador previo

        # extraer datos de los productos detectados (hasta max_results)
        for item in productos_actuales[:max_results]:
            try:
                titulo = item.find_element(By.CLASS_NAME, "pod-subTitle").text.strip()
                # busca el elemento con clase 'pod-subTitle' dentro del item y obtiene su texto (título del producto)
                precio = item.find_element(By.CSS_SELECTOR, "li[data-event-price] span").text.strip()
                # busca el precio usando un selector más específico (li con atributo data-event-price -> span interno)
                link = item.get_attribute("href")            # obtiene la url del producto desde el atributo href del enlace

                resultados.append({
                    "titulo": titulo,                        # agrega el título al diccionario
                    "precio": precio,                        # agrega el precio al diccionario
                    "link": link,                            # agrega el link al diccionario
                    "origen": "Falabella"                    # marca el origen (útil si mezclas resultados de varias tiendas)
                })

                print(f"{titulo}\n{precio} — {link}\n")      # imprime en consola el producto extraído (lectura humana)

            except Exception:
                continue                                      # si falla la extracción de un campo para ese item, ignoramos ese item y seguimos

    except Exception as e:
        print("❌ Error general:", e)                         # captura cualquier error inesperado en la ejecución y lo muestra

    finally:
        driver.quit()                                        # cierra el navegador siempre (asegura liberar recursos aunque haya error)

    return resultados                                        # devuelve la lista de resultados recopilados


# Hiraoka

In [69]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus

def scrape_hiraoka(query, max_pages=3):
    productos = []

    for page in range(1, max_pages + 1):
        url = f"https://hiraoka.com.pe/gpsearch/?p={page}&q={quote_plus(query)}"
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")

        items = soup.find_all("div", class_="product-item-info")
        if not items:
            break

        for item in items:
            nombre_tag = item.find("a", class_="product-item-link")
            titulo = nombre_tag.text.strip() if nombre_tag else None
            link = nombre_tag['href'] if nombre_tag else None

            precio_tag = item.find("span", class_="price-wrapper")
            if precio_tag:
                # Limpiar espacios no separables u otros caracteres raros
                precio = precio_tag.text.strip().replace("\xa0", " ")
            else:
                precio = "No disponible"

            productos.append({
                "titulo": titulo,
                "precio": precio,
                "link": link,
                "origen": "Hiraoka"
            })

    return productos


# Mercado Libre 

In [ ]:
import requests
from bs4 import BeautifulSoup


def scrape_mercadolibre(query):
    print(f"\n📦 Resultados de MercadoLibre para: {query}")

    url = f"https://listado.mercadolibre.com.pe/{query.replace(' ', '-')}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("❌ Error al obtener resultados de MercadoLibre")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    items = soup.select("li.ui-search-layout__item")

    resultados = []

    for item in items[:10]:  # Máximo 10 productos
        try:
            titulo = item.select_one("a.poly-component__title").get_text(strip=True)
            precio = item.select_one("div.poly-price__current span.andes-money-amount__fraction").get_text(strip=True)
            link = item.select_one("a.poly-component__title")["href"]

            resultados.append({
                "titulo": titulo,
                "precio": f"S/ {precio}",
                "link": link,
                "origen": "MercadoLibre"
            })

            print(f"{titulo}\nS/ {precio} — {link}\n")

        except Exception as e:
            print("⚠️ Producto con error:", e)
            continue

    return resultados



# Ripley

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random

# Configuración de Selenium
options = Options()
options.add_argument("--headless")  # ejecuta sin abrir ventana
options.add_argument("window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.get("https://simple.ripley.com.pe/search/monitor?sort=relevance_desc&page=1")

# Espera que cargue la página
time.sleep(random.uniform(3, 6))

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Lista donde guardaremos los resultados
resultados = []

productos = soup.find_all("div", class_="catalog-product-details")

for producto in productos:
    titulo = producto.find("div", class_="catalog-product-details__name").text.strip()
    
    marca_tag = producto.find("div", class_="brand-logo")
    marca = marca_tag.text.strip() if marca_tag else "GENÉRICO"
    
    precio_tag = producto.find("li", class_="catalog-prices__offer-price")
    precio = precio_tag.text.strip() if precio_tag else "No disponible"
    
    # Extraer link del producto
    link_tag = producto.find("a", href=True)
    link = "https://simple.ripley.com.pe" + link_tag['href'] if link_tag else "No disponible"
    
    envio_tag = producto.find("div", class_="emblem--dispatch")
    envio = envio_tag.text.strip() if envio_tag else "No disponible"
    
    # Guardar en diccionario
    resultado = {
        "titulo": titulo,
        "marca": marca,
        "precio": precio,
        "link": link,
        "origen": "Ripley",
        "envio": envio
    }
    
    resultados.append(resultado)

# Mostrar resultados
for r in resultados:
    print(r)


{'titulo': "MONITOR FULL HD IPS BOWMANN BW-22MF GAMER 100 HZ PLANO 22'' 1920X1080", 'marca': 'BOWMANN', 'precio': 'S/ 270', 'link': 'No disponible', 'origen': 'Ripley', 'envio': 'No disponible'}
{'titulo': "MONITOR FULL HD IPS BOWMANN BW-24MF GAMER 100 HZ PLANO 24'' 1920X1080", 'marca': 'BOWMANN', 'precio': 'S/ 300', 'link': 'No disponible', 'origen': 'Ripley', 'envio': 'No disponible'}
{'titulo': "MONITOR PORTÁTIL PEPPER JOBS USB-C DE 15.6' XTENDVIZ XV1610F PANTALLA FULL HD IPS ALTAVOCES DUALES HDMI OTG PARA SAMSUNG", 'marca': 'GLOBAL SHOPEX', 'precio': 'S/ 1,059', 'link': 'No disponible', 'origen': 'Ripley', 'envio': 'Envío gratis internacional'}
{'titulo': "MONITOR PORTÁTIL ACTUALIZADO DE 15.6' IPS HDR 1920X1080 FHD EYE CARE SCREEN USB C MONITOR DE JUEGOS PANTALLA", 'marca': 'GLOBAL SHOPEX', 'precio': 'S/ 449', 'link': 'No disponible', 'origen': 'Ripley', 'envio': 'Envío gratis internacional'}
{'titulo': "MONITOR PORTÁTIL COCOPAR ACTUALIZADO 17.3' 1080P FHD IPS HDR SRGB FREESYNC USB

# Coolboox

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random

def scrape_coolbox(query):
    url = f"https://www.coolbox.pe/audifonos?_q={query}&map=ft"

    # Configuración de Selenium
    options = Options()
    options.add_argument("--headless")
    options.add_argument("window-size=1920,1080")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    time.sleep(random.uniform(3, 6))  # Espera aleatoria para cargar la página

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    resultados = []
    productos = soup.find_all("div", class_="coolboxpe-store-search-0-x-galleryItem")

    for producto in productos:
        # Título
        titulo_tag = producto.find("section", class_="vtex-product-summary-2-x-container")
        titulo = titulo_tag.get("aria-label") if titulo_tag else "No disponible"

        # Marca
        marca_tag = producto.find("span", class_="vtex-store-components-3-x-productBrandName")
        marca = marca_tag.text.strip() if marca_tag else ""

        # Agregar marca al final del título
        if marca:
            titulo = f"{titulo} - {marca}"

        # Precio
        precio_tag = producto.find("span", class_="vtex-product-price-1-x-sellingPriceValue")
        if precio_tag:
            precio_texto = precio_tag.text.strip()
            # Limpiar texto y convertir a float
            precio = precio_texto.replace("S/", "").replace("\xa0", "").replace(",", "")
            try:
                precio = float(precio)
            except:
                precio = None
        else:
            precio = None

        # Link
        link_tag = producto.find("a", href=True)
        link = "https://www.coolbox.pe" + link_tag['href'] if link_tag else "No disponible"

        # Envío
        envio_tag = producto.find("p", class_="coolboxpe-store-logistic-0-x-shippingTagDay")
        envio = envio_tag.text.strip() if envio_tag else "No disponible"

        resultados.append({
            "titulo": titulo,
            "precio": precio,
            "link": link,
            "origen": "Coolbox"
        })

    return resultados


In [6]:
scrape_coolbox("teclado")

[{'titulo': 'Product Airpods Pro 2da gen cancelación de ruido, resistente al agua IPX4, duración máx. 30 horas con estuche de carga, MagSafe, blanco - Apple',
  'precio': 1049.9,
  'link': 'https://www.coolbox.pe/audifonos-bluetooth-tw-apple-airpods-pro-2da-gen-6h-tipoc-mtjv3am-a/p',
  'origen': 'Coolbox'},
 {'titulo': 'Product Audífonos con cancelación de ruido JBL Tour One M2 micrófono incorporado, máx. 50 horas, control de música y llamadas, negro - JBL',
  'precio': 499.9,
  'link': 'https://www.coolbox.pe/audifonos-cancelacion-ruido-jbl-tour-one-m2-50h-jbltouronem2blk/p',
  'origen': 'Coolbox'},
 {'titulo': 'Product Airpods (4ta Gen) resistente al agua IP54, duración máx. 30 horas con estuche de carga, control táctil, blanco - Apple',
  'precio': 599.9,
  'link': 'https://www.coolbox.pe/audifonos-bt-tws-apple-airpods-4ta-gen-30h-mxp63am-a/p',
  'origen': 'Coolbox'},
 {'titulo': 'Product Airpods (4ta Gen) cancelación activa de ruido, resistente al agua IP54, duración máx. 30 horas 

# Plaza Vea

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re  # <- para limpiar el precio

def scrape_plazavea_selenium(query, max_scrolls=10, max_results=50):
    print(f"\n🛍️ Resultados de Plaza Vea para: {query}")

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--start-maximized")

    driver = webdriver.Chrome(options=options)
    resultados = []

    try:
        url = f"https://www.plazavea.com.pe/search/?_query={query.replace(' ', '%20')}"
        driver.get(url)

        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.Showcase__content"))
        )

        prev_count = 0
        for i in range(max_scrolls):
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
            time.sleep(3)

            productos_actuales = driver.find_elements(By.CSS_SELECTOR, "div.Showcase__content")
            print(f"🔄 Scroll {i+1}: {len(productos_actuales)} productos detectados")

            if len(productos_actuales) == prev_count:
                break
            prev_count = len(productos_actuales)

        for item in productos_actuales[:max_results]:
            try:
                titulo_tag = item.find_element(By.CSS_SELECTOR, "button.Showcase__name")
                titulo = titulo_tag.text.strip() if titulo_tag else None

                precio_tag = item.find_element(By.CSS_SELECTOR, "div.Showcase__salePrice span.price")
                precio_texto = precio_tag.text.strip() if precio_tag else "No disponible"

                # 🔹 Limpiar el precio: dejar solo el número
                precio = re.sub(r"[^\d.,]", "", precio_texto)  # elimina todo excepto dígitos y punto/coma

                link_tag = item.find_element(By.CSS_SELECTOR, "a.Showcase__link")
                link = link_tag.get_attribute("href") if link_tag else None

                resultados.append({
                    "titulo": titulo,
                    "precio": precio,
                    "link": link,
                    "origen": "Plaza Vea"
                })

                print(f"{titulo}\n{precio} — {link}\n")

            except Exception:
                continue

    except Exception as e:
        print("❌ Error general:", e)

    finally:
        driver.quit()

    # Eliminar duplicados por link
    seen = set()
    resultados_unicos = []
    for r in resultados:
        if r['link'] not in seen:
            resultados_unicos.append(r)
            seen.add(r['link'])

    return resultados_unicos
